## Localisation du Label

In [1]:
### Importations de modules

import pytesseract
from PIL import Image
import cv2
import numpy as np



In [2]:
img = cv2.imread('/home/marco/Projet_Localisation_des_Signatures/cin/1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
for i, text in enumerate(data['text']):
    if "Signature et Empreintes" in text:

        x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
        signature_roi = img[y+h+10:y+h+100, x:x+w]  # Ajuster la hauteur selon besoin
        cv2.imwrite('signature_roi.jpg', signature_roi)

        roi = cv2.cvtColor(signature_roi, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(roi, 150, 255, cv2.THRESH_BINARY_INV)  # Inverser pour noir sur blanc
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

## Approche par Deep Learning

In [14]:
from ultralytics import YOLO

# Charger un modèle pré-entraîné 
model = YOLO("yolov8s.pt")  

# Entraînement
results = model.train(
    data="dataset.yaml",
    epochs=100,
    batch=8,
    imgsz=640,
    device="0"  # "0" pour GPU, "cpu" sinon
)

Ultralytics 8.3.160 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1060, 6066MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tru

train: Scanning /home/marco/Projet_Localisation_des_Signatures/Dataset/labels/train.cache... 7 images, 0 backgrounds, 6 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]

train: /home/marco/Projet_Localisation_des_Signatures/Dataset/images/train/01.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/marco/Projet_Localisation_des_Signatures/Dataset/images/train/02.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/marco/Projet_Localisation_des_Signatures/Dataset/images/train/03.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/marco/Projet_Localisation_des_Signatures/Dataset/images/train/04.jpg: ignoring corrupt image/label: Label class 4 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/marco/Projet_Localisation_des_Signatures/Dataset/images/train/05.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/marco/Projet_Localisation_des_Signatures/Datase

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 235.7±0.0 MB/s, size: 1349.9 KB)


val: Scanning /home/marco/Projet_Localisation_des_Signatures/Dataset/labels/val... 0 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<00:00, 4378.19it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     31/100      2.38G      1.072      2.294      1.624          2        640: 100%|██████████| 1/1 [00:00<00:00,  5.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 45.06it/s]

                   all          1          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



/home/marco/.env/lib/python3.10/site-packages/ultralytics/utils/metrics.py:765: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/home/marco/.env/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


KeyboardInterrupt: 

## Test Visuel 